In [ ]:
# Trabalho 3 - Image Restoration
# SCC0251 - Image Processing (01/2021)
# Fabiana Dalacqua Mendes - 9894536
# Pedro Henrique Nieuwenhoff - 10377729
import numpy as np
import imageio
import math

In [ ]:
def restore(img,F,gamma):
    """
    This function ...
    
    Parameters
    ----------
    img : uint8 array
        The degraded image array
    F : int
        TThe type of filter to perform image restoration (1 (denoising) or 2 (constrained l.s.))
    gamma : int
        The parameter used by either filter
    """
    
    def adaptive_denoising(coord,n,denoise):
        res_img = img
        return res_img
        
    def constrained_least_squares(n,sigma):
        res_img = img
        return res_img
   
    if F == 1: 
         # getting coordinates of a flat rectangle in the image
        coord = np.array([float(xi) for xi in input().split()])
        n = int(input()) # getting the size of the filter
        denoise = str(input().rstrip()) # getting the denoise mode (average or robust)
        
        return adaptive_denoising(coord,n,denoise)
    elif F == 2:
        n = int(input()) # getting the size of the filter
        sigma = float(input())
        
        return constrained_least_squares(n,sigma)

In [ ]:
def clipping(img):
    """
    This function ...
    
    Parameters
    ----------
    img : uint8 array
        The image array
    """
    print()

In [ ]:
def rmse_compare(f,g):
    """
    This function compare images through Root Mean Squared Error (RMSE). 
    The higher the SRE, the greater the difference betweenthe images.
    
    Parameters
    ----------
    img_a : array
        The first image array
    img_b : array
        The second image array
    """
    
    # getting images resolution 
    N,M = f.shape # both images have the same size
    
    # converting the matrices to int32, allowing negative difference
    f = f.astype(np.int32)
    g = g.astype(np.int32)
    
    # computing rsme
    rmse = np.sum(np.square(f-g))
    rmse = math.sqrt(rmse / (N*M))

    return rmse

In [ ]:
def image_restoration():
    """
    This function ...
    
    Inputs (in this order)
    ----------------------
    ref_img : string
        The filename for the reference image
    deg_img : string
        The filename for the degraded image
    F : int
        The type of filter to perform image restoration (1 (denoising) or 2 (constrained l.s.))
    gamma : int
        The parameter used by either filter
    """
    
    # getting input parameters
    ref = str(input().rstrip())
    deg = str(input().rstrip())
    F = int(input())
    gamma = float(input())
    
    # reading images as uint8 array
    ref_img = imageio.imread(ref)
    deg_img = imageio.imread(deg)
    
    res_img = restore(deg_img,F,gamma) # applying restoration method in degraded image
    print(rmse_compare(ref_img, res_img)) # comparing through rmse value

In [ ]:
if __name__ == "__main__":
    image_restoration()